# SETTINGS

In [1]:
############ LIBRARIES

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('dark_background')
%matplotlib inline

import os
import time
import datetime
import random
import multiprocessing
import pickle

import scipy.stats

import gc
gc.enable()

import warnings
warnings.filterwarnings('ignore')

In [2]:
############ HELPER FUNCTIONS

import functions
import importlib
importlib.reload(functions)
from functions import *

In [3]:
############ RANDOMNESS

# seed function
def seed_everything(seed = 23):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    
# set seed
seed = 23
seed_everything(seed)

# IMPORT

In [4]:
############ DATA IMPORT

# id data
train_id = pd.read_csv('../input/train_identity.csv')
test_id  = pd.read_csv('../input/test_identity.csv')

# transaction data
train_tr = pd.read_csv('../input/train_transaction.csv')
test_tr  = pd.read_csv('../input/test_transaction.csv')

# check dimensions
print(train_id.shape)
print(train_tr.shape)
print('-'*15)
print(test_id.shape)
print(test_tr.shape)

(144233, 41)
(590540, 394)
---------------
(141907, 41)
(506691, 393)


In [5]:
# check data
train_id.head()

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987004,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NotFound,NaN,-480.0,New,NotFound,166.0,NaN,542.0,144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,Android 7.0,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
1,2987008,-5.0,98945.0,NaN,NaN,0.0,-5.0,NaN,NaN,NaN,NaN,100.0,NotFound,49.0,-300.0,New,NotFound,166.0,NaN,621.0,500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,iOS 11.1.2,mobile safari 11.0,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device
2,2987010,-5.0,191631.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,100.0,NotFound,52.0,NaN,Found,Found,121.0,NaN,410.0,142.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Found,Found,NaN,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,Windows
3,2987011,-5.0,221832.0,NaN,NaN,0.0,-6.0,NaN,NaN,NaN,NaN,100.0,NotFound,52.0,NaN,New,NotFound,225.0,NaN,176.0,507.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,NaN,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,NaN
4,2987016,0.0,7460.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0,0.0,100.0,NotFound,NaN,-300.0,Found,Found,166.0,15.0,529.0,575.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Found,Found,Mac OS X 10_11_6,chrome 62.0,24.0,1280x800,match_status:2,T,F,T,T,desktop,MacOS


In [6]:
# check data
train_tr.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,13.0,NaN,NaN,NaN,0.0,T,T,T,M2,F,T,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,M0,T,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,

# MERGER

In [7]:
# target variable
target = 'isFraud'
test_tr[target]  = np.nan

In [8]:
############ ALIGN DATA SETS

# align columns
train_tr = train_tr.reindex(sorted(train_tr.columns), axis = 1)
train_id = train_id.reindex(sorted(train_id.columns), axis = 1)
test_tr  = test_tr.reindex(sorted(test_tr.columns),   axis = 1)
test_id  = test_id.reindex(sorted(test_id.columns),   axis = 1)

# check equalty
print(np.all(train_tr.columns == test_tr.columns))
print(np.all(train_id.columns == test_id.columns))

True
True


In [9]:
############ MERGE TRAIN & TEST

# rbind columns
df_tr = pd.concat([train_tr, test_tr], axis = 0)
df_id = pd.concat([train_id, test_id], axis = 0)
print(df_tr.shape)
print(df_id.shape)

# clear memory
del train_tr, test_tr
del train_id, test_id

(1097231, 394)
(286140, 41)


In [10]:
# merge ID and transactions
df = pd.merge(df_tr, df_id, on = 'TransactionID', how = 'left')
print(df.shape)
del df_tr, df_id

(1097231, 434)


# PROCESSING

### COMPRESS

In [11]:
# compress data
df = reduce_mem_usage(df)

Mem. usage decreased to 1226.38 Mb (66.3% reduction)


### DROP IRRELEVANT FEATURES

In [12]:
# remove columns with a single value
print(df.shape)
df = df.loc[:, df.nunique(dropna = False) != 1]
print(df.shape)

(1097231, 434)
(1097231, 434)


In [13]:
# remove irrelevant columns
print(df.shape)
drops = []
for var in drops:
    del df[var]
print(df.shape)

(1097231, 434)
(1097231, 434)


### MISSING VALUES

In [14]:
# check missings
count_missings(df)

,Total,Percent
id_24,1087744,99.135369
id_25,1087060,99.073030
id_26,1087021,99.069476
id_07,1087017,99.069111
id_08,1087017,99.069111
id_21,1087013,99.068747
id_22,1087000,99.067562
id_23,1087000,99.067562
id_27,1087000,99.067562
dist2,1023168,93.250008


### VARIABLE TYPES

In [15]:
# check data types
df.dtypes

C1       float16
C10      float16
C11      float16
C12      float16
C13      float16
C14      float16
C2       float16
C3       float16
C4       float16
C5       float16
C6       float16
C7       float16
C8       float16
C9       float16
D1       float16
D10      float16
D11      float16
D12      float16
D13      float16
D14      float16
D15      float16
D2       float16
D3       float16
D4       float16
D5       float16
D6       float16
D7       float16
D8       float16
D9       float16
M1        object
          ...   
id_09    float16
id_10    float16
id_11    float16
id_12     object
id_13    float16
id_14    float16
id_15     object
id_16     object
id_17    float16
id_18    float16
id_19    float16
id_20    float16
id_21    float16
id_22    float16
id_23     object
id_24    float16
id_25    float16
id_26    float16
id_27     object
id_28     object
id_29     object
id_30     object
id_31     object
id_32    float16
id_33     object
id_34     object
id_35     object
id_36     obje

In [16]:
# check distributions
df.describe()

,C1,C10,C11,C12,C13,C14,C2,C3,C4,C5,C6,C7,C8,C9,D1,D10,D11,D12,D13,D14,D15,D2,D3,D4,D5,D6,D7,D8,D9,TransactionAmt,TransactionDT,TransactionID,V1,V10,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V11,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V12,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V13,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V14,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V15,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V16,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V17,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V18,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V19,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V2,V20,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V21,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V22,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V23,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V24,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V25,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V26,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V27,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V28,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V29,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V3,V30,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V31,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V32,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V33,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,V34,V35,V36,V37,V38,V39,V4,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V5,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V6,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V7,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V8,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V9,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,addr1,addr2,card1,card2,card3,card5,dist1,dist2,isFraud,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_13,id_14,id_17,id_18,id_19,id_20,id_21,id_22,id_24,id_25,id_26,id_32
count,1097228.0,1097228.0,1097228.0,1097228.0,1092483.0,1097228.0,1097228.0,1097228.0,1097228.0,1097228.0,1097228.0,1097228.0,1097228.0,1097228.0,1089931.0,1008664.0,641426.0,133971.0,185336.0,177381.0,996049.0,581665.0,631211.0,851458.0,563015.0,197970.0,99050.0,149264.00000,149264.000000,1.097231e+06,1.097231e+06,1.097231e+06,641426.0,641426.0,1096917.0,1096917.0,1096917.0,1096917.0,1096917.0,1096917.0,1096917.0,1096917.0,1096917.0,1096917.0,641426.0,1096917.0,1096917.0,1096917.0,1096917.0,1096917.0,1096917.0,1096917.0,1096917.0,1096917.0,1096917.0,1008569.0,1096917.0,1096917.0,1096917.0,1096917.0,1096917.0,1096917.0,1.096917e+06,1.096917e+06,1.096917e+06,1096917.0,1008569.0,1.096917e+06,1.096917e+06,1.096917e+06,1.096917e+06,1.096917e+06,1.096917e+06,1.096917e+06,1.096917e+06,157730.0,157730.0,1008569.0,157730.0,157730.0,157730.0,158006.0,158006.0,158006.0,157730.0,157730.0,157730.0,157730.0,1008569.0,158006.0,158006.0,158006.0,157730.0,157730.0,157730.0,157730.0,157730.0,157730.0,158006.000000,1008569.0,1.580060e+05,157730.0,157730.0,157730.0,158006.000000,158006.000000,158006.000000,276365.0,276365.0,276194.0,1008569.0,276194.0,276194.0,276365.0,276365.0,276194.0,276194.0,276365.0,276365.0,276365.0,276365.0,1008569.0,276194.0,276365.0,276365.0,276365.0,276194.0,276194.0,276365.0,276365.0,276194.0,276194.0,1008569.0,276365.0,276365.0,276365.0,276365.0,276194.0,276194.0,276365.0,276194.0,276194.0,276365.0,641426.0,1008569.0,276194.0,276194.0,2.763650e+05,2.763650e+05,2.763650e+05,276365.0,276365.0,276365.000000,276194.0,276194.0,1008569.0,276194.0,276365.000000,276365.000000,276365.000000,276365.000000,276365.000000,276365.000000,257158.0,257158.0,257158.0,1008569.0,278732.0,278732.0,278732.0,257158.0,257158.0,257158.0,257158.0,278732.0,257158.0,257158.0,1008569.0,257158.0,257158.0,257158.0,257158.0,278732.0,257158.0,257158.0,257158.0,278732.0,278732.0,1008569.0,257158.0,257158.0,257158.0,257158.0,257158.0,278732.0,257158.0,257158.0,257158.0,257158.0,1008569.0,278732.0,278732.0,257158.0,257158.0,257158.0,278732.0,278732.0,257158.0,257158.

In [17]:
# check value counts
facs = [f for f in df.columns if df[f].dtype == "object"]
for fac in facs:
    print('--------------------------------')
    print(fac + ': ' + str(df[fac].nunique()) + ' unique values')
    print('--------------------------------')
    print(df[fac].value_counts(normalize = True, dropna = False).head(2))
    print('--------------------------------')
    print('')

--------------------------------
M1: 2 unique values
--------------------------------
T      0.591886
NaN    0.408063
Name: M1, dtype: float64
--------------------------------

--------------------------------
M2: 2 unique values
--------------------------------
T      0.536189
NaN    0.408063
Name: M2, dtype: float64
--------------------------------

--------------------------------
M3: 2 unique values
--------------------------------
T      0.472320
NaN    0.408063
Name: M3, dtype: float64
--------------------------------

--------------------------------
M4: 3 unique values
--------------------------------
NaN    0.473181
M0     0.326084
Name: M4, dtype: float64
--------------------------------

--------------------------------
M5: 2 unique values
--------------------------------
NaN    0.601618
F      0.218874
Name: M5, dtype: float64
--------------------------------

--------------------------------
M6: 2 unique values
--------------------------------
F    0.382265
T    0.318528
N

In [18]:
# class imbalance
df[target].value_counts(normalize = True, dropna = True)

0.0    0.96501
1.0    0.03499
Name: isFraud, dtype: float64

# EXPORT

In [ ]:
# export data
df.to_pickle("../input/data.pkl")
df.shape